In [2]:
#  Importing Libraries & Modules
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [14]:
# Importing Dataset
path = "Stock_Prediction_Dataset.csv"
raw_data = pd.read_csv(path)
raw_data.head()

Table 1
NaN NaN        NaN    NaN    NaN   NaN   NaN    NaN                               NaN
    Date       Open   High   Low   Last  Close  Total Trade Quantity  Turnover (Lacs)
    2018-09-28 234.05 235.95 230.2 233.5 233.75 3069914                       7162.35
    2018-09-27 234.55 236.8  231.1 233.8 233.25 5082859                      11859.95
    2018-09-26 240    240    232.5 235   234.25 2240909                        5248.6

In [18]:
raw_data.drop(col)

KeyError: 'Table 1'